In [10]:
import cairo
from IPython.display import Image, display
import math
from io import BytesIO
import numpy as np

def disp(draw_func):
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, 200, 200)
    ctx = cairo.Context(surface)
    draw_func(ctx, 200, 200)
    with BytesIO() as fileobj:
        surface.write_to_png(fileobj)
        display(Image(fileobj.getvalue(), width=200))


In [27]:

def draw_aircraft(
    surface_width: int,
    surface_height: int,
    # Wing parameters
    wingspan: float,
    sweep_angle_deg: float,
    chord_positions: list[float],
    chord_lengths: list[float],
    # Fuselage parameters
    fuselage_width: float,
    fuselage_length: float,
    cockpit_length: float,
    tail_length: float,
    scale: float = 1.0
) -> None:
    surface = cairo.ImageSurface(cairo.FORMAT_RGB24, surface_width, surface_height)
    ctx = cairo.Context(surface)
    
    # Set white background
    ctx.set_source_rgb(1, 1, 1)
    ctx.paint()
    
    # Set black line color
    ctx.set_source_rgb(0, 0, 0)
    ctx.set_line_width(2)
    
    # Center the drawing
    ctx.translate(surface_width / 2, surface_height / 2)
    
    # [Wing drawing code remains exactly the same as before]
    sweep_angle = math.radians(sweep_angle_deg)
    
    # Draw right half of wing
    ctx.move_to(0, 0)
    x_tip = wingspan/2 * scale
    y_tip = x_tip * math.tan(sweep_angle)
    ctx.line_to(x_tip, y_tip)
    
    points = list(zip(chord_positions, chord_lengths))
    points.sort(reverse=True)
    
    for pos, chord in points:
        x = pos * scale
        y = x * math.tan(sweep_angle) + chord * scale
        ctx.line_to(x, y)
    
    ctx.line_to(0, chord_lengths[0] * scale)
    ctx.close_path()
    
    # Draw left half
    ctx.move_to(0, 0)
    ctx.line_to(-x_tip, y_tip)
    for pos, chord in points:
        x = -pos * scale
        y = -x * math.tan(sweep_angle) + chord * scale
        ctx.line_to(x, y)
    ctx.line_to(0, chord_lengths[0] * scale)
    
    # Stroke the wing
    ctx.stroke()
    
    # Calculate wing centroid (approximate as midpoint of root chord)
    wing_centroid_x = 0
    wing_centroid_y = chord_lengths[0] * scale / 2
    
    # Draw fuselage on top with corrected orientation
    f_width = fuselage_width * scale
    f_length = fuselage_length * scale
    c_length = cockpit_length * scale
    t_length = tail_length * scale
    
    total_length = f_length + c_length + t_length
    
    # Fill fuselage with white to cover wing lines
    ctx.set_source_rgb(1, 1, 1)
    
    # Main body (rectangle)
    ctx.rectangle(
        -f_width/2,
        wing_centroid_y - f_length/2,
        f_width,
        f_length
    )
    ctx.fill()
    
    # Cockpit (half ellipse)
    ctx.save()
    ctx.translate(0,wing_centroid_y - f_length/2)
    ctx.scale(c_length/2, f_width/2)
    ctx.arc(0, 0, 1, -math.pi,0)  # Draw right half
    ctx.restore()
    ctx.fill()
    
    # Tail (triangle)
    ctx.move_to(wing_centroid_y + f_length/2, -f_width/2)
    ctx.line_to(wing_centroid_y + f_length/2 + t_length, 0)
    ctx.line_to(wing_centroid_y + f_length/2, f_width/2)
    ctx.fill()
    
    # Now draw the outlines in black
    ctx.set_source_rgb(0, 0, 0)
    
    # Main body outline
    ctx.rectangle(
        -f_width/2,
        wing_centroid_y - f_length/2,
        f_width,
        f_length,
    )
    ctx.stroke()
    
    # Cockpit outline (half ellipse)
    ctx.save()
    ctx.translate(0,wing_centroid_y - f_length/2)
    ctx.scale(f_width/2,c_length/2)
    ctx.arc(0, 0, 1,-math.pi,0)  # Draw right half
    ctx.restore()
    ctx.stroke()
    
    # Tail outline
    ctx.move_to(wing_centroid_y + f_length/2, -f_width/2)
    ctx.line_to(wing_centroid_y + f_length/2 + t_length, 0)
    ctx.line_to(wing_centroid_y + f_length/2, f_width/2)
    ctx.stroke()
    
    # Save to PNG
    surface.write_to_png("aircraft.png")

In [25]:
wingspan = 315  # units
root_chord = 32. * 2
sweep_angle = 37.5  # degrees
chord_positions = np.array([0, .4, 1]) * wingspan/2  # distances from centerline
#chord_lengths = np.array([1, .75, .5, .2]) * root_chord # distances from centerline
chord_lengths = np.array([1,
                          1-.4/np.tan(sweep_angle*np.pi/180.),
                          0.1])  * root_chord    # chord lengths at each position

# Fuselage parameters
fuselage_width = 33
fuselage_length = 175
cockpit_length = 45
tail_length = 50

# Create drawing
draw_aircraft(
    800, 600,
    wingspan, sweep_angle, chord_positions, chord_lengths,
    fuselage_width, fuselage_length, cockpit_length, tail_length,
    scale=2
)